## Library imports

In [1]:
import numpy as np
import pandas as pd

## Dataset Import

In [2]:
alist = pd.read_csv('../data/Peter32.csv')

In [4]:
alist.head()

,Peak,k0,S
0,1,76.707539,5.74
1,2,3394.799565,8.31
2,3,18.915846,4.60
3,4,320.537733,6.92
4,5,376999.824156,12.60


## Useful functions definition

In [5]:
def sig(height, mu):
    return np.sqrt((mu^2) * height)

In [7]:
def plerror(analyte_list):
    pass